In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np

In [ ]:
f=open('/content/drive/MyDrive/AlphaFold/design.pdb')   #入力するPDBファイル
g=open('/content/drive/MyDrive/AlphaFold/design_nma.pdb',"w") #出力する構造変化予測ファイル
thr=10 #バネを張る閾値を指定(パラメータ)
xyz=[]
linef=[]
linel=[]

for line in f:  #PDBのAチェーンのCα原子の座標を読み込む
    if line[0:4]=="ATOM":
        if line[13:15]=="CA":
            if line[21:22]=="A":
                x=float(line[30:38])
                y=float(line[38:46])
                z=float(line[46:54])
                xyz.append([x,y,z])
                first=(line[:30])
                last=(line[54:]).replace('\n','')
                linef.append(first)
                linel.append(last)
f.close()
N=len(xyz)

d=np.zeros((N,N,))

for i in range(N):  #Cα原子間の距離を求める
    for j in range(N):
        d[i][j]=((xyz[i][0]-xyz[j][0])**2+(xyz[i][1]-xyz[j][1])**2+(xyz[i][2]-xyz[j][2])**2)**(0.5)

H=np.zeros((3*N,3*N,))

for i in range(N):  #ヘシアン行列(バネ定数に相当する行列)を計算する
    for j in range(N):
        if d[i][j]<=thr and i!=j:
            for k in range(3):
                for l in range(3):
                    H[3*i+k][3*j+l]=(-1)*(xyz[i][k]-xyz[j][k])*(xyz[i][l]-xyz[j][l])/d[i][j]**2
                    H[3*i+k][3*i+l]=H[3*i+k][3*i+l]+(-1)*H[3*i+k][3*j+l]

eigenvalue,eigenvector=np.linalg.eig(H) #ヘシアン行列の固有値問題を解く
eigen_id=np.argsort(eigenvalue) #固有値に小さい順の番号をつける

mode=6 #構造変化のモード番号(6から3N-1までの数値)を指定(パラメータ)
amplitude=40 #振幅を指定(パラメータ)
frames=50 #ムービーのフレーム数を指定

for j in range(frames): #予測した構造変化を出力。ここではサイン関数で動かす(お好きな関数に改造してください)。
    t=((j/frames)*10)*np.pi
    print("MODEL"+'{:>9d}'.format(j+1),file=g)
    for i in range(N):
        xt=xyz[i][0]+amplitude*eigenvector[3*i+0][eigen_id[mode]]*np.sin(t)
        yt=xyz[i][1]+amplitude*eigenvector[3*i+1][eigen_id[mode]]*np.sin(t)
        zt=xyz[i][2]+amplitude*eigenvector[3*i+2][eigen_id[mode]]*np.sin(t)
        print(linef[i]+'{:>8.3f}'.format(xt.real)+'{:>8.3f}'.format(yt.real)+'{:>8.3f}'.format(zt.real)+linel[i],file=g)
    print("ENDMDL",file=g)

g.close()
